# <center>  Model Creat Dataframe diplomescommuness [DIP]<br/></center>
<a name="TOP"></a>
- dev : nyx
- Fichier CSV : Diplomes_csv/Diplomes_csv/diplomescommunes.csv</a>

ALERTES 98 lignes à Nan 
Faire les plus proches voisins apres FUSION ?
    
# Description Fichiers : 
Capital immo pour 2022 , 2012 , 2002 , 1992
avec pour chaque année :
    4 capital immo
    5 prix du bien

# Synthese des Actions :
Plus de 1000 communes non renseignees flag 1 pour BL_bug
creation de lignes avec que des 0
  
**<u>MENU</u>** 
- <a href="#Import">P0 : Import CSV</a>
- <a href="#EDA">P1 : Mini EDA</a>
- <a href="#Nan">P2 : Traitement des Nan</a>
- <a href="#TRANSCO">P3 : Filtres et/ou calculs</a>
- <a href="#EXPORT">P4 : EXPORT (/DATA/EXPORT_CSV/final_xxxxxxx.csv)</a>





PREFIXE FICHIER : KP1
Structure :

    0 dep
    1 nomdep
    2 codecommune
    3 nomcommune
    4 KP1_capitalimmo22
    5 KP1_prixbien22
    6 KP1_capitalimmo12
    7 KP1_prixbien12
    8 KP1_capitalimmo02
    9 KP1_prixbien02
    10 KP1_capitalimmo92
    11 KP1_prixbien92
    12 DBL_bug (flag à 1 poour les lignes ajoutées avec des Nan A TRAITER)




In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import zipfile
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## <a name="Import"></a>Partie 0 : Import CSV&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <a href="#TOP">&uarr;</a>

In [2]:
def df_creat(filename, df_dataframe):  
    df = df_dataframe[df_dataframe['filename'] == filename]
    if len(df) == 1:
        row = df.iloc[0]
        file = row['path']
        if os.path.exists(file):
            print(file)
            df_from_csv = pd.read_csv(file, encoding='utf-8', low_memory=False)
            return df_from_csv
        else:
            print('FICHIER NON TROUVE', file)
            return pd.DataFrame()
    else:
        print(f"filename: '{filename}' return size: {len(df)}")
        return pd.DataFrame()

In [3]:
csv_file_path = 'df_filtred.csv'
# Use pandas to read the CSV file into a DataFrame with UTF-8 encoding
df_cat = pd.read_csv(csv_file_path, encoding='utf-8')

In [4]:
df_cat.head(1)

,category,filename,comment,size,depart,district,commune,canton,yyyy,path,codecommune
0,Taille_agglo_commune_csv,codescommunes1999,codes,0,0,0,1,0,1999,../../ProjetCommun/DATA_SOURCES/HCP/Taille_agg...,1


In [5]:
file_path = '../DATA/list_codecommune_leg2022_50inscrits.txt'

# Open the text file and read lines into a list
with open(file_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file.readlines()]

# Print or manipulate the list of lines as needed
print(lines[0:10], len(lines))

['01001', '01002', '01004', '01005', '01006', '01007', '01008', '01009', '01010', '01011'] 33830


In [6]:
df = df_creat('diplomescommunes', df_cat)

../../ProjetCommun/DATA_SOURCES/HCP/Diplomes_csv/Diplomes_csv/diplomescommunes.csv


## <a name="EDA"></a>Partie 1 : Mini EDA &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <a href="#TOP">&uarr;</a>

In [7]:
df.shape

(37985, 1163)

In [8]:
df.head(5)

,dep,nomdep,codecommune,nomcommune,nodiph1962,bach1962,suph1962,nodipf1962,bacf1962,supf1962,...,perbac1955,persup1955,perbac1956,persup1956,perbac1957,persup1957,perbac1958,persup1958,perbac1959,persup1959
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,121.0,4.0,0.0,116.0,0.0,2.0,...,0.263747,0.258736,0.263747,0.258736,0.263747,0.258736,0.263747,0.258736,0.263747,0.258736
1,01,AIN,01002,ABERGEMENT-DE-VAREY,40.0,0.0,2.0,19.0,0.0,0.0,...,0.263809,0.642062,0.263809,0.642062,0.263809,0.642062,0.263809,0.642062,0.263809,0.642062
2,01,AIN,01003,AMAREINS,32.0,0.0,0.0,35.0,0.0,0.0,...,0.047666,0.092142,0.047666,0.092142,0.047666,0.092142,0.047666,0.092142,0.047666,0.092142
3,01,AIN,01004,AMBERIEU-EN-BUGEY,2241.0,89.0,40.0,2430.0,57.0,16.0,...,0.475746,0.405178,0.475746,0.405178,0.475746,0.405178,0.475746,0.405178,0.475746,0.405178
4,01,AIN,01005,AMBERIEUX-EN-DOMBES,181.0,0.0,2.0,163.0,0.0,2.0,...,0.140375,0.416333,0.140375,0.416333,0.140375,0.416333,0.140375,0.416333,0.140375,0.416333


In [9]:
codecommune_listdf = df['codecommune'].tolist()
codecommune_listdf[1], len(codecommune_listdf)

('01002', 37985)

In [10]:
df['codecommune'].nunique(),len(df)

(37985, 37985)

In [11]:
absent = [x for x in lines if x not in codecommune_listdf]
len (absent)

27

In [12]:
dico={}
list_year=[2022,2012,2002,1992]
for c in df :
    value =df[c].iloc[0]
    try:
        last4char = int(c[-4:])
        if last4char > 0 and last4char in list_year :
            dico[c] = value
            print (c)
    except ValueError as e:
        dico[c] = value
print (dico)
    

nodiph1992
nodiph2002
nodiph2012
nodiph2022
bach1992
bach2002
bach2012
bach2022
suph1992
suph2002
suph2012
suph2022
nodipf1992
nodipf2002
nodipf2012
nodipf2022
bacf1992
bacf2002
bacf2012
bacf2022
supf1992
supf2002
supf2012
supf2022
nodip1992
nodip2002
nodip2012
nodip2022
bac1992
bac2002
bac2012
bac2022
sup1992
sup2002
sup2012
sup2022
pbac1992
perbac1992
pbac2002
perbac2002
pbac2012
perbac2012
pbac2022
perbac2022
psup1992
persup1992
psup2002
persup2002
psup2012
persup2012
psup2022
persup2022
{'dep': '01', 'nomdep': 'AIN', 'codecommune': '01001', 'nomcommune': 'ABERGEMENT-CLEMENCIAT', 'nodiph1992': 171.0, 'nodiph2002': 194.0, 'nodiph2012': 163.0, 'nodiph2022': 155, 'bach1992': 9.0, 'bach2002': 29.0, 'bach2012': 42.0, 'bach2022': 62, 'suph1992': 14.0, 'suph2002': 32.0, 'suph2012': 65.0, 'suph2022': 77, 'nodipf1992': 178.0, 'nodipf2002': 195.0, 'nodipf2012': 149.0, 'nodipf2022': 163, 'bacf1992': 6.0, 'bacf2002': 26.0, 'bacf2012': 42.0, 'bacf2022': 62, 'supf1992': 11.0, 'supf2002': 29.0, 's

In [13]:
df.shape

(37985, 1163)

In [14]:
index_col_sup = df.shape[1]

In [15]:
list_yyyy = [2022,2012,2002,1992]
list_indic = ['nodip','nodipf','nodiph',
              'bac','bacf','bach',
              'sup','supf','supf',             
              'pbac','psup']
list_col = df.columns.tolist()
for yyyy in list_yyyy :
    for indic in list_indic :
        col_source = str(indic) + str(yyyy)
        if col_source in list_col :
            yy=str(yyyy)[-2:]
            new_col='DIP_'+ str(indic) +'_'+ str(yy)
            df[new_col] = df[col_source]
df.shape

(37985, 1203)

In [16]:
df = df.drop(df.columns[4:index_col_sup], axis=1)
df.head(2)

,dep,nomdep,codecommune,nomcommune,DIP_nodip_22,DIP_nodipf_22,DIP_nodiph_22,DIP_bac_22,DIP_bacf_22,DIP_bach_22,...,DIP_nodip_92,DIP_nodipf_92,DIP_nodiph_92,DIP_bac_92,DIP_bacf_92,DIP_bach_92,DIP_sup_92,DIP_supf_92,DIP_pbac_92,DIP_psup_92
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,319,163,155,124,62,62,...,349.0,178.0,171.0,15.0,6.0,9.0,24.0,11.0,0.100515,0.061856
1,01,AIN,01002,ABERGEMENT-DE-VAREY,57,17,40,102,55,47,...,89.0,54.0,35.0,14.0,4.0,10.0,12.0,2.0,0.226087,0.104348


In [17]:
for i,c in enumerate(df) : 
    print (i,c)

0 dep
1 nomdep
2 codecommune
3 nomcommune
4 DIP_nodip_22
5 DIP_nodipf_22
6 DIP_nodiph_22
7 DIP_bac_22
8 DIP_bacf_22
9 DIP_bach_22
10 DIP_sup_22
11 DIP_supf_22
12 DIP_pbac_22
13 DIP_psup_22
14 DIP_nodip_12
15 DIP_nodipf_12
16 DIP_nodiph_12
17 DIP_bac_12
18 DIP_bacf_12
19 DIP_bach_12
20 DIP_sup_12
21 DIP_supf_12
22 DIP_pbac_12
23 DIP_psup_12
24 DIP_nodip_02
25 DIP_nodipf_02
26 DIP_nodiph_02
27 DIP_bac_02
28 DIP_bacf_02
29 DIP_bach_02
30 DIP_sup_02
31 DIP_supf_02
32 DIP_pbac_02
33 DIP_psup_02
34 DIP_nodip_92
35 DIP_nodipf_92
36 DIP_nodiph_92
37 DIP_bac_92
38 DIP_bacf_92
39 DIP_bach_92
40 DIP_sup_92
41 DIP_supf_92
42 DIP_pbac_92
43 DIP_psup_92


In [18]:
df.isna().sum()

dep                 0
nomdep            243
codecommune         0
nomcommune          0
DIP_nodip_22        0
DIP_nodipf_22       0
DIP_nodiph_22       0
DIP_bac_22          0
DIP_bacf_22         0
DIP_bach_22         0
DIP_sup_22          0
DIP_supf_22         0
DIP_pbac_22      2763
DIP_psup_22      2763
DIP_nodip_12     2697
DIP_nodipf_12    2697
DIP_nodiph_12    2697
DIP_bac_12       2697
DIP_bacf_12      2697
DIP_bach_12      2697
DIP_sup_12       2697
DIP_supf_12      2697
DIP_pbac_12      2708
DIP_psup_12      2708
DIP_nodip_02     1416
DIP_nodipf_02    1416
DIP_nodiph_02    1416
DIP_bac_02       1416
DIP_bacf_02      1416
DIP_bach_02      1416
DIP_sup_02       1416
DIP_supf_02      1416
DIP_pbac_02      1425
DIP_psup_02      1425
DIP_nodip_92     1429
DIP_nodipf_92    1429
DIP_nodiph_92    1429
DIP_bac_92       1429
DIP_bacf_92      1429
DIP_bach_92      1429
DIP_sup_92       1429
DIP_supf_92      1429
DIP_pbac_92      1436
DIP_psup_92      1436
dtype: int64

In [19]:
df.shape

(37985, 44)

In [20]:
df['codecommune'].nunique(),len(df)

(37985, 37985)

## <a name="TRANSCO"></a>Partie 3 : Filtres et/ou calculs&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#TOP">&uarr;</a>

In [21]:
df_filtered = df[df['codecommune'].isin(lines)]
df_filtered.shape,df.shape

((33803, 44), (37985, 44))

In [22]:
codecommune_listdf_filtered= df_filtered['codecommune'].tolist()
codecommune_listdf_filtered[1], len(codecommune_listdf_filtered)

('01002', 33803)

In [23]:
absent = [x for x in lines if x not in codecommune_listdf_filtered]
len (absent)

27

In [24]:
absent

['13201',
 '13202',
 '13203',
 '13204',
 '13205',
 '13206',
 '13207',
 '13208',
 '13209',
 '13210',
 '13211',
 '13212',
 '13213',
 '13214',
 '13215',
 '13216',
 '69380',
 '69381',
 '69382',
 '69383',
 '69384',
 '69385',
 '69386',
 '69387',
 '69388',
 '69389',
 '75056']

Toujours les memes 27 communes qui posent pb !!!

['13201', '13202','13203','13204','13205','13206','13207', '13208', '13209', '13210', '13211', '13212', '13213', '13214', '13215', '13216', '69281', '69282', '69283', '69284', '69285',
 '69286', '69287', '69288', '69289', '69380', '75056']



In [25]:
df_filtered['DBL_bug'] = 0

In [26]:
df_filtered.head(1)

,dep,nomdep,codecommune,nomcommune,DIP_nodip_22,DIP_nodipf_22,DIP_nodiph_22,DIP_bac_22,DIP_bacf_22,DIP_bach_22,...,DIP_nodipf_92,DIP_nodiph_92,DIP_bac_92,DIP_bacf_92,DIP_bach_92,DIP_sup_92,DIP_supf_92,DIP_pbac_92,DIP_psup_92,DBL_bug
0,01,AIN,01001,ABERGEMENT-CLEMENCIAT,319,163,155,124,62,62,...,178.0,171.0,15.0,6.0,9.0,24.0,11.0,0.100515,0.061856,0


In [27]:
new_row = ['*', '*', 'code', '*'] + [np.nan] * 40 +[1]
len (new_row)

45

In [28]:
len(df_filtered.columns)

45

In [29]:
for i,code in enumerate(absent) :
    new_row = ['*', '*', code, '*'] + [np.nan] * 40 + [1]
    row_df = pd.DataFrame([new_row], columns=df_filtered.columns)
    df_filtered = pd.concat([df_filtered, row_df], ignore_index=True)
df_filtered.shape

(33830, 45)

## <a name="EXPORT"></a>Partie 4 : EXPORT&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="#TOP">&uarr;</a>



In [30]:
path_export='../DATA/EXPORT_CSV/final_DIP_diplomescommunes.csv'

In [31]:
df_filtered.to_csv(path_export, sep=';', index=False)
print(f'DataFrame exported to: {path_export}')

DataFrame exported to: ../DATA/EXPORT_CSV/final_DIP_diplomescommunes.csv
